In [1]:
import pandas as pd
import numpy as np
import numpy.matlib
import math 
from numpy import linalg as LA

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
prison = pd.read_csv('/content/drive/My Drive/IS_HW2/prison_dataset.csv').sample(frac = 1, random_state =1).sample(frac = 1, random_state =1).sample(frac = 1, random_state =1)
header = prison.columns
test = prison.iloc[0:int(15424/4),:]
train = prison.iloc[int(15424/4):-1,:]
train

,Fiscal Year Released,Recidivism Reporting Year,Race - Ethnicity,Age At Release,Convicting Offense Classification,Convicting Offense Type,Convicting Offense Subtype,Main Supervising District,Release Type,Part of Target Population,Recidivism - Return to Prison numeric
11613,2015,2018,White,<45,Other Felony,Other,Other,5JD,Parole,Yes,1
14466,2015,2018,White,<45,Other Felony,Violent,Other,3JD,Discharged End of Sentence,Yes,0
12799,2015,2018,Black,<45,Other Felony,Drug,Other,3JD,Parole,No,0
3708,2013,2016,White,>45,D Felony,Drug,Other,3JD,Parole,Yes,1
8596,2015,2018,Black,<45,Other Felony,Other,Other,3JD,Discharged End of Sentence,No,0
...,...,...,...,...,...,...,...,...,...,...,...
2679,2013,2016,White,<45,Other Felony,Other,Other,3JD,Discharged End of Sentence,No,1
259,2010,2013,Black,<45,D Felony,Other,Other,5JD,Discharged End of Sentence,Yes,1
10138,2015,2018,White,<45,D Felony,Other,Other,3JD,Discharged End of Sentence,Yes,0
295,2010,2013,White,<45,Other Felony,Other,Other,3JD,Parole,Yes,1


In [4]:
train.iloc[2]['Release Type']

'Parole'

In [5]:
prison_tot = prison.to_numpy()
prison_tot.shape

(15424, 11)

In [6]:
Age_At_Release = prison['Age At Release'].to_numpy()
Convicting_Offense_Classification = prison['Convicting Offense Classification'].to_numpy()
Convicting_Offense_Subtype = prison['Convicting Offense Subtype'].to_numpy()
Convicting_Offense_Type = prison['Convicting Offense Type'].to_numpy()
Fiscal_Year_Released = prison['Fiscal Year Released'].to_numpy()
Main_Supervising_District = prison['Main Supervising District'].to_numpy()
Part_of_Target_Population = prison['Part of Target Population'].to_numpy()
Race_Ethnicity = prison['Race - Ethnicity'].to_numpy()
Recidivism_Return_to_Prison_numeric = prison['Recidivism - Return to Prison numeric'].to_numpy()
Recidivism_Reporting_Year = prison['Recidivism Reporting Year'].to_numpy()
Release_Type = prison['Release Type'].to_numpy()

In [7]:
UNQ_Age_At_Release = prison['Age At Release'].unique()
UNQ_Convicting_Offense_Classification = prison['Convicting Offense Classification'].unique()
UNQ_Convicting_Offense_Subtype = prison['Convicting Offense Subtype'].unique()
UNQ_Convicting_Offense_Type = prison['Convicting Offense Type'].unique()
UNQ_Fiscal_Year_Released = prison['Fiscal Year Released'].unique()
UNQ_Main_Supervising_District = prison['Main Supervising District'].unique()
UNQ_Part_of_Target_Population = prison['Part of Target Population'].unique()
UNQ_Race_Ethnicity = prison['Race - Ethnicity'].unique()
UNQ_Recidivism_Return_to_Prison_numeric = prison['Recidivism - Return to Prison numeric'].unique()
UNQ_Recidivism_Reporting_Year = prison['Recidivism Reporting Year'].unique()
UNQ_Release_Type = prison['Release Type'].unique()


In [8]:
UNQ_Convicting_Offense_Type

array(['Violent', 'Drug', 'Other'], dtype=object)

In [9]:
def entropy(probablities):
    ts = np.sum(probablities)
    probablities = probablities/ts
    temp = 0
    for i in range(probablities.shape[0]):
        if probablities[ i ] == 0 :
            return 0
            break
        else:
            temp += - probablities[ i ] * np.log2( probablities[ i ] )
    return temp

In [10]:
def match_unq_leaf(index, prison):
    end = prison.shape[1] -1
    temp = prison.iloc[:,[index,end]].to_numpy()
    unq = prison.iloc[:,index].unique()
    matched = np.zeros((unq.shape[0],2))
    for i in range(unq.shape[0]):
        for row in range(temp.shape[0]):
            if temp[row][0] == unq[i]:
                if temp[row][1] ==1:
                    matched[i, 1] += 1
                else:
                    matched[i, 0] += 1
    return matched

In [11]:
def cal_branch_prob(arr):
    vs = np.sum(arr , axis = 1)
    tot = np.sum(vs)
    return vs/tot

In [12]:
def gain(prison_tot):
    tot_gain = np.zeros(prison_tot.shape[1]-1)
    entropy_tot = entropy(np.sum(match_unq_leaf(prison_tot.shape[1]-1, prison_tot ), axis = 0))
    for index in range(prison_tot.shape[1] - 1):
        matched = match_unq_leaf(index,prison_tot)
        matched_prob = cal_branch_prob(matched)
        temp = 0
        for i in range(matched_prob.shape[0]):
            temp += matched_prob[ i ] * entropy( matched[i, :] )
        tot_gain[ index ] = entropy_tot - temp
    return tot_gain


In [14]:
def create_branch(header, gains, data):
    best_feature = np.argmax(gains)
    yu = header[best_feature]
    atr = data.iloc[:,best_feature].unique()
    root_data = {}
    for root in atr:
        root_data[root] = data.where(root == data[header[best_feature]]).dropna()
        root_data[root] = root_data[root].drop([header[best_feature]], axis = 1)
    temp=np.delete(header, best_feature)
    return temp, root_data, yu

In [17]:
def is_one(a,o):
    a=a.to_numpy()
    length = a.shape[0]
    b = np.count_nonzero(a)
    if o :
        return b==length
    else:
        return b==0

In [18]:
def greater_one(a):
    a=a.to_numpy()
    length = a.shape[0]
    b = np.count_nonzero(a)
    return b>length/2

In [20]:
class Node():
    def __init__(self, node_data ,  header, branches = [], leaf_atr= None , gain = None, split_feature = None):
        self.data = node_data
        self.branches = branches
        self.header = header
        self.leaf_atr = leaf_atr
        self.gain = gain
        self.split_feature = split_feature
    def reset(self):
        self.branches = []


In [21]:
a=[]
def ID3(train, header, depth):
    node = Node(train, header)
    node.reset()
    if is_one(node.data.iloc[:,[-1]],1):
        #print('here')
        node.leaf_atr = 1
        return node

    if is_one(node.data.iloc[:,[-1]],0):
        #print('2')
        node.leaf_atr = 0
        return node

    if (node.header.shape[0] == 1) | (depth == 0):
        if greater_one(node.data.iloc[:,[-1]]):
            node.leaf_atr = 1
        else:
            node.leaf_atr = 0
        return node
    
    node.gain = gain(node.data)
    branch_header, branch_data, lop = create_branch( node.header, node.gain, node.data )
    node.split_feature = lop
    for feature in branch_data.keys():
        node.branches.append( ID3(branch_data[feature], branch_header, depth - 1) )
    return node

In [24]:
Layer_1 = []
res = []
def heirac(root):
    
    if len(root.branches) == 0 :
        a=[]
        a.append(root.split_feature)
        return a
    else:
        t=[]
        for i in range(len(root.branches)):
            temp=[]
            temp.append(heirac(root.branches[i]))
            temp.append(root.split_feature)
            t.append(temp)
        return t
        

In [26]:
def surfe(data):
    fiscal = data['Fiscal Year Released']
    age = data['Age At Release']
    convict = data['Convicting Offense Type']
    district = data['Main Supervising District']
    release = data['Release Type']
    bin = data['Recidivism - Return to Prison numeric']
    
    if fiscal == 2015:
        if age == '<45':
            if convict == 'Violent':
                return bin == 0
            if convict == 'Drug':
                return bin == 0
            if convict == 'Other':
                return bin == 0
        if age == '>45':
            if district == '3JD':
                return bin == 0
            if district == '5JD':
                return bin == 0
    elif fiscal == 2013:
        if release == 'Discharged End of Sentence':
            if convict == 'Violent':
                return bin == 1
            if convict == 'Drug':
                return bin == 0
            if convict == 'Other':
                return bin == 1
        if release == 'Parole':
            if district == '3JD':
                return bin == 1
            if district == '5JD':
                return bin == 1
    elif fiscal == 2010:
        return bin == 1


In [27]:
 prison['Convicting Offense Type'].unique()

array(['Violent', 'Drug', 'Other'], dtype=object)

In [28]:
 prison['Age At Release'].unique()

array(['<45', '>45'], dtype=object)

In [29]:
 prison['Fiscal Year Released'].unique()

array([2015, 2013, 2010])

In [30]:
prison['Main Supervising District'].unique()

array(['3JD', '5JD'], dtype=object)

In [31]:
prison['Release Type'].unique()

array(['Discharged End of Sentence', 'Parole'], dtype=object)

In [50]:
def test_rf(test, forest, ind):
    pred = np.zeros((test.shape[0], trees.shape[0]), dtype = int)
    for i in range(trees.shape[0]):
        queri = test.iloc[:,ind[i,:-1]].to_dict(orient = "records")
        for j in range(test.shape[0]):
            pred[j,i] = predict(queri[j],trees[i])
    result = np.zeros(test.shape[0], dtype = int)
    for j in range(test.shape[0]):
        result[j] = mod(pred[j,:])
    return result

In [55]:
def mod(a):
    counts = np.bincount(a)
    return np.argmax(counts)


In [56]:
def random_forest(train, header, depth, tree_num, attr_num):
    a = np.zeros((tree_num, attr_num+1), dtype=int)
    temp = np.arange(10)
    for i in range(tree_num):
        np.random.shuffle(temp)
        a[i,0:attr_num] = temp[:attr_num]
        a[i,attr_num] = 10
    trees = np.empty(tree_num, dtype = object)
    for i in range(tree_num):
        trees[i] = deep(ID3(train.iloc[:,list(a[i,:])] ,np.take(header, list(a[i,:])),depth), train)
    return trees,a

In [40]:
 
def deep(a,data):  
    if len(a.branches) == 0:
        return a.leaf_atr
    else:
        tree = { a.split_feature : { } }
        for i in range(len(a.branches)):
            subtree = deep(a.branches[i],data)
            tree[a.split_feature][data[a.split_feature].unique()[i]] = subtree
    return tree

In [32]:
def testing(test):
    leng=test.shape[0]
    correct = 0
    for i in range(leng):
        if surfe(test.iloc[i]):
            correct += 1
    return correct/leng

In [13]:
gains = gain(train)
for i in range(10):
    print('Gain of '+ "{:s}".format(str(header[i])) + ' is: ' +"{:3s}".format(' ') + str(gains[i]))

Gain of Fiscal Year Released is:    0.21805471844518043
Gain of Recidivism Reporting Year is:    0.21805471844518043
Gain of Race - Ethnicity is:    0.0001064139946544973
Gain of Age At Release is:    0.0056997109028152915
Gain of Convicting Offense Classification is:    0.0005237713713839298
Gain of Convicting Offense Type is:    0.0037213415523231763
Gain of Convicting Offense Subtype is:    0.00115878411292214
Gain of Main Supervising District is:    0.009217681338014927
Gain of Release Type is:    0.010023069949074426
Gain of Part of Target Population is:    0.016313781585013398


In [22]:
root = ID3(train, header, 3)

In [62]:
rt = deep(root, test)
pred = np.zeros(test.shape[0], dtype = int)
queri = test.iloc[:,:-1].to_dict(orient = "records")
for j in range(test.shape[0]):
    pred[j] = predict(queri[j],rt)
confusion_mat = np.zeros((2,2) , dtype = int)
for i in range(result.shape[0]):
    confusion_mat[test.iloc[i,-1],pred[i]] += 1
print(confusion_mat)

[[1482  214]
 [1042 1118]]


In [33]:
acc=testing(test)*100
print('Accuracy for ID3 and max depth of 3 is : '+ str(acc))

Accuracy for ID3 and max depth of 3 is : 70.77282157676349


In [45]:
testin(test, deep(ID3(train,header,3), test))

The prediction accuracy is:  67.42738589211619 %


In [41]:
a = deep(root,prison)
a

{'Fiscal Year Released': {2010: 1,
  2013: {'Age At Release': {'<45': {'Convicting Offense Type': {'Drug': 0,
      'Other': 1,
      'Violent': 1}},
    '>45': {'Main Supervising District': {'3JD': 1, '5JD': 1}}}},
  2015: {'Release Type': {'Discharged End of Sentence': {'Convicting Offense Type': {'Drug': 0,
      'Other': 0,
      'Violent': 0}},
    'Parole': {'Main Supervising District': {'3JD': 0, '5JD': 0}}}}}}

In [42]:
def predict(query,tree,default = 0):
    for key in list(query.keys()):
        if key in list(tree.keys()):
            try:
                result = tree[key][query[key]] 
            except:
                return default
            result = tree[key][query[key]]
            if isinstance(result,dict):
                return predict(query,result)
            else:
                return result

In [43]:
def testin(data,tree):
    queries = data.iloc[:,:-1].to_dict(orient = "records")
    predicted = pd.DataFrame(columns=["predicted"]) 
    for i in range(len(data)):
        predicted.loc[i,"predicted"] = predict(queries[i],tree,0) 
    print('The prediction accuracy is: ',(np.sum(predicted["predicted"].to_numpy() == data.iloc[:,-1].to_numpy())/len(data))*100,'%')

In [44]:
deep(ID3(train,header,3), prison)

{'Fiscal Year Released': {2010: 1,
  2013: {'Age At Release': {'<45': {'Convicting Offense Type': {'Drug': 0,
      'Other': 1,
      'Violent': 1}},
    '>45': {'Main Supervising District': {'3JD': 1, '5JD': 1}}}},
  2015: {'Release Type': {'Discharged End of Sentence': {'Convicting Offense Type': {'Drug': 0,
      'Other': 0,
      'Violent': 0}},
    'Parole': {'Main Supervising District': {'3JD': 0, '5JD': 0}}}}}}

In [66]:
trees,a=random_forest(train,header,3,15, 2)

In [67]:
trees.shape

(15,)

In [68]:
result = test_rf(test, trees, a)

In [69]:
print('The prediction accuracy is: ',(np.sum(result== test.iloc[:,-1].to_numpy())/len(test))*100,'%')

The prediction accuracy is:  71.47302904564316 %


In [74]:
confusion_mat = np.zeros((2,2) , dtype = int)
for i in range(result.shape[0]):
    confusion_mat[test.iloc[i,-1],result[i]] += 1


In [75]:
print(confusion_mat)

[[1433  263]
 [ 837 1323]]


In [63]:
from sklearn import preprocessing
test_data = test.to_numpy()
train_data = train.to_numpy()
le = preprocessing.LabelEncoder()
a = []
for i in range(11):
    le.fit(train_data.T[i])
    a.append(le.transform(train_data.T[i]))

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
b = []
for i in range(11):
    le.fit(test_data.T[i])
    b.append(le.transform(test_data.T[i]))

from sklearn import tree
from sklearn.metrics import confusion_matrix
clf = tree.DecisionTreeClassifier(criterion="entropy", random_state=0, max_depth=3)
clf = clf.fit(np.array(a).T[:,:-1],np.array(a).T[:,-1])

y_pred = clf.predict(np.array(b).T[:,:-1])
print("Accuracy: " + str(np.sum(y_pred == test_data[:,-1])/y_pred.shape[0]) )
s =test_data[:,-1].shape
z=np.zeros(s,dtype = int)
for i in range(test_data.shape[0]):
    z[i] = int(test_data[i,-1])
print("Confusion matrix is : \n" , confusion_matrix(z,y_pred))

Accuracy: 0.7147302904564315
Confusion matrix is : 
 [[1433  263]
 [ 837 1323]]


In [64]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
clf = RandomForestClassifier(criterion="entropy", random_state=0, max_depth=3)
clf = clf.fit(np.array(a).T[:,:-1],np.array(a).T[:,-1])
y_pred = clf.predict(np.array(b).T[:,:-1])
print("Accuracy: " + str(np.sum(y_pred == test_data[:,-1])/y_pred.shape[0]) )
#

Accuracy: 0.7152489626556017


In [65]:
s =test_data[:,-1].shape
a=np.zeros(s,dtype = int)
for i in range(test_data.shape[0]):
    a[i] = int(test_data[i,-1])
#a = test_data[:,-1]
print("Confusion matrix is : \n" , confusion_matrix(a,y_pred))


Confusion matrix is : 
 [[1396  300]
 [ 798 1362]]
